In [9]:
import pandas as pd
import numpy as np
import plotly
import re
import math
import textblob
from textblob import TextBlob
from textblob import Blobber
from textblob.wordnet import Synset

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# To do: 
- Separate by "or" and ";" also! - Done!
- *Polarity check before meaning check because of stop words processing should be better 'v' (kinda done)*

#### Reading the pre-cleaned data

In [12]:
flower_data = pd.read_csv("../Data/General/flower_cleaned.csv", error_bad_lines=False, delimiter=";")

In [13]:
flower_data.sample(10)

,Flower,Meaning,SimilarIndex30,SimilarIndex20,SimilarIndex40
181,Laurel,success,[256],"[233, 256, 358]","[233, 256, 358]"
349,Violet [blue],faithfulness,[],[],[]
306,Snowdrop,hope,"[338, 341]","[325, 338, 341]","[325, 338, 341]"
192,Lily [orange],desire,"[198, 228, 287, 293, 306, 338, 341]","[193, 197, 198, 207, 213, 215, 228, 236, 243, ...","[193, 197, 198, 207, 213, 215, 228, 236, 243, ..."
200,Lime blossom,fornication,[],[],[]
173,Hydrangea,heartfelt gratitude for being understood,[],"[178, 187, 257, 260, 263, 286, 295, 299, 315, ...","[178, 187, 257, 260, 263, 286, 295, 299, 315, ..."
29,Azalea,temperance,[],[],[]
133,Dandelion,overcoming hardship,[],"[144, 151, 153, 154, 160, 174, 178, 187, 206, ...","[144, 151, 153, 154, 160, 174, 178, 187, 206, ..."
74,Carnation [yellow],rejection,[],[218],[218]
99,Coreopsis,always cheerful,[],[336],[336]


---
## - NLP towards network graph building

---
#### Using AFINN DICTIONARY for sentiment analysis

In [41]:
path = "../External/afinn-master/afinn/data/AFINN-111.txt"
word_data_afinn = pd.read_csv(path, delimiter="\t", header=None, names=["Word", "Score"])

In [42]:
word_data_afinn.sample(n = 10)

,Word,Score
962,fatality,-3
849,enlightens,2
1907,retreat,-1
2137,strengthening,2
1204,hopefully,2
1964,scapegoat,-2
370,chances,2
1947,sadden,-2
1296,infected,-2
792,drunk,-2


In [43]:
def afinn_score(sentence):
    split_sentence = sentence.split(" ")
    total_score = word_data_afinn.Score[word_data_afinn.Word.isin(split_sentence)].sum()
    return total_score

In [44]:
flower_data['AfinnScore'] = flower_data.Meaning.apply(afinn_score)

In [45]:
flower_data.sample(n=10)

,Flower,Meaning,SimilarIndex30,SimilarIndex20,SimilarIndex40,AfinnScore
50,Bird of paradise flower,good perspective,"[86, 154, 174, 325]","[51, 86, 91, 92, 122, 132, 154, 157, 174, 203,...",[174],3
144,Forget-me-not,true love,"[151, 153, 160, 169, 178, 187, 193, 206, 207, ...","[151, 153, 160, 166, 168, 169, 173, 178, 187, ...","[260, 344, 187, 151, 153, 160]",5
357,Witch-hazel,a magic spell,[],[361],[],0
81,Carnation [mauve],dreams of fantasy,"[167, 180, 212, 247, 252, 350]","[85, 88, 92, 98, 99, 100, 101, 108, 119, 125, ...","[252, 350, 212]",1
96,China aster,fidelity,"[177, 349]","[177, 313, 349]","[177, 349]",0
179,Jonquil,return my affection,"[197, 281]","[187, 197, 206, 228, 236, 257, 260, 263, 270, ...",[],3
12,Anemone,anticipation,[],"[306, 338]",[],1
330,Tulip [purple],rebirth,[],[],[],0
172,Hydrangea,heartlessness (negative),[194],[194],[],0
265,Rose [white],innocence,[268],[268],[268],0


---
#### Using TEXTBLOB for semantic meaning comparison

In [19]:
wiki = TextBlob("Python is a high-level, general-purpose programming language.")
wiki.tags

[('Python', 'NNP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('high-level', 'JJ'),
 ('general-purpose', 'JJ'),
 ('programming', 'NN'),
 ('language', 'NN')]

In [20]:
wiki.sentiment
TextBlob("my regrets follow you to the grave").sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

Sentiment(polarity=-0.1, subjectivity=0.2)

In [21]:
a = wiki.words
a

WordList(['Python', 'is', 'a', 'high-level', 'general-purpose', 'programming', 'language'])

In [22]:
# Programming
a[5].lemmatize('v')

'program'

Get the highest comparison score for each synsets of a word
Limit to nouns words

In [23]:
textblob.Word("like").definitions
textblob.Word("Love").definitions

neverendsyn = textblob.Word("like").synsets
lovesyn = textblob.Word("Love").synsets

print("Similarity:", lovesyn[6].path_similarity(neverendsyn[0]))

['a similar kind',
 'a kind of person',
 'prefer or wish to do something',
 'find enjoyable or agreeable',
 'be fond of',
 'feel about or towards; consider, evaluate, or regard',
 'want to have',
 'resembling or similar; having the same or some of the same characteristics; often used in combination',
 'equal in amount or value',
 'having the same or similar characteristics',
 'conforming in every respect']

['a strong positive emotion of regard and affection',
 'any object of warm affection or devotion; ',
 'a beloved person; used as terms of endearment',
 'a deep feeling of sexual desire and attraction',
 'a score of zero in tennis or squash',
 'sexual activities (often including sexual intercourse) between two people',
 'have a great affection or liking for',
 'get pleasure from',
 'be enamored or in love with',
 'have sexual intercourse with']

Similarity: 0.08333333333333333


In [24]:
textblob.Word("thriftiness").definitions
textblob.Word("economy").definitions

['frugality in the expenditure of money or resources']

['the system of production and distribution and consumption',
 'the efficient use of resources',
 'frugality in the expenditure of money or resources',
 'an act of economizing; reduction in cost']

In [25]:
stop_words = set(stopwords.words('english'))

In [26]:
def clean_stopwords(sentence, stopwords):
    clean_sentence = []
    for word in sentence.split(" "):
        if word not in stopwords:
            clean_sentence.append(word)
    return " ".join(clean_sentence)

In [27]:
clean_stopwords("my regrets follow you to the grave", stop_words)
ablob = TextBlob("Love between two woman.")
ablob.words

'regrets follow grave'

WordList(['Love', 'between', 'two', 'woman'])

In [28]:
# Returns the highest similarity score between two sentences (with 1 being very similar, 0 being different).
def compare_meaning_sentence(sentence1, sentence2, stopwords):
    blob1 = TextBlob(clean_stopwords(sentence1, stopwords))
    blob2 = TextBlob(clean_stopwords(sentence2, stopwords))
    nb_words = len(blob1.words) + len(blob2.words)
    
    similarity_score = 0
    for word1 in blob1.words:
        for word2 in blob2.words:
            similarity_score += compare_meaning_word(word1, word2)
    return similarity_score / nb_words
                             
# Returns the highest similarity score between two words (with 1 being very similar, 0 being different).                             
def compare_meaning_word(word1, word2):
    highest_similarity_score = 0
    
    for synset1 in word1.synsets:
        for synset2 in word2.synsets:
            similarity_score = synset1.path_similarity(synset2)
            
            if(similarity_score != None and similarity_score > highest_similarity_score):
                #print(synset1, synset2, similarity_score)
                highest_similarity_score = similarity_score
                
    return highest_similarity_score



In [33]:
numbers = {}
numbers['a'] = 4
numbers['b'] = 2
numbers['c'] = 7
sorted(numbers, key = numbers.get, reverse=True)
#sorted(numbers.values())

['c', 'a', 'b']

In [39]:
# Builds the edge network connection with respect to sentences similarities.
# The similarity_threshold refers to the lower limit of similarity score that is considered similar.
def index_flower_similarity(similarity_threshold, stopwords = stop_words, link_limit = 6):
    similarities = []
    
    for i in range(0, len(flower_data)):
        similar_indexes = []
        similar_index_score = {}
        
        for j in range(i+1, len(flower_data)):
            #print("indexes:", i, j)
            score = compare_meaning_sentence(flower_data['Meaning'].iloc[i], flower_data['Meaning'].iloc[j], stop_words)
            
            if score > similarity_threshold:
                similar_indexes.append(j)
                similar_index_score[j] = score
                
        similar_index_score = sorted(similar_index_score, key = similar_index_score.get, reverse=True)
        del similar_index_score[link_limit:]
        
        similarities.append(similar_index_score)
        
    return similarities

#### Adding to the csv the similarity nodes with a threshold of 0.4

In [46]:
#network_links = index_flower_similarity(0.4)
flower_data['SimilarIndex40'] = pd.Series(network_links)
flower_data.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)

In [23]:
flower_data.head(10)

,Flower,Meaning
0,Acacia,secret love
1,Acanthus,art
2,Aconite,misanthropy
3,Agrimony,thankfulness
4,Aloe,grief
5,Almond,promise
6,Amaranth (Globe),immortal love
7,Amaranth,immortality
8,Amaryllis,pride
9,Ambrosia,love is reciprocated


#### Adding to the csv the similarity nodes with a threshold of 0.3

In [24]:
network_links = index_flower_similarity(0.3)
flower_data['SimilarIndex30'] = pd.Series(network_links)

In [25]:
flower_data.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)

#### Adding to the csv the similarity nodes with a threshold of 0.2

In [26]:
network_links = index_flower_similarity(0.2)
flower_data['SimilarIndex20'] = pd.Series(network_links)
flower_data.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)